**Klasifikacija bijelih krvnih stanica**

## Spajanje na google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Potrebni paketi

In [ ]:
import matplotlib.pyplot as plt     # plotting 
import numpy as np                  # Matrix image operations
import cv2                          # Computer vision
import glob                         # File path manipulation
import os

### Putanje do mapa

In [81]:
input_path_eosino = '/content/drive/My Drive/Projects/Blood_Cells_Dataset/blood_cell_dataset/dataset2-master/dataset2-master/images/TRAIN/EOSINOPHIL/'
input_path_lympho = '/content/drive/My Drive/Projects/Blood_Cells_Dataset/blood_cell_dataset/dataset2-master/dataset2-master/images/TRAIN/LYMPHOCYTE/'
input_path_mono =   '/content/drive/My Drive/Projects/Blood_Cells_Dataset/blood_cell_dataset/dataset2-master/dataset2-master/images/TRAIN/MONOCYTE/'
input_path_neutro = '/content/drive/My Drive/Projects/Blood_Cells_Dataset/blood_cell_dataset/dataset2-master/dataset2-master/images/TRAIN/NEUTROPHIL/'

output_path_eosino = '/content/drive/My Drive/Projects/Blood_Cells_Dataset/blood_cell_dataset/dataset2-master/dataset2-master/images/TRAIN_PROCESSED/EOSINOPHIL/'
output_path_lympho = '/content/drive/My Drive/Projects/Blood_Cells_Dataset/blood_cell_dataset/dataset2-master/dataset2-master/images/TRAIN_PROCESSED/LYMPHOCYTE/'
output_path_mono =   '/content/drive/My Drive/Projects/Blood_Cells_Dataset/blood_cell_dataset/dataset2-master/dataset2-master/images/TRAIN_PROCESSED/MONOCYTE/'
output_path_neutro = '/content/drive/My Drive/Projects/Blood_Cells_Dataset/blood_cell_dataset/dataset2-master/dataset2-master/imagesTRAIN_PROCESSED/NEUTROPHIL/'

## Funkcije za pretprocesiranje slika

In [ ]:
def create_directory(path):
    """ Creates empty folder if folder doesn't  exist.

    Args:
        path (string): Relative or absolute path for creating a folder
    """
    %mkdir '/content/drive/My Drive/Projects/Blood_Cells_Dataset/blood_cell_dataset/dataset2-master/dataset2-master/images/TRAIN_PROCESSED/'
    
    if not os.path.exists(path):
      %mkdir '$path'
      print('Creating folder structure at:', path)


def remove_if_exists(path):
    """ Removes all files on given path for fresh results
    on every script run.

    Args:
        path (string): Input path for file removal.
    """

    for item in glob.glob(path):
        if os.path.exists(path):
            %rm -rf '$path'
            print('Path ' + path + 'existed, so it was deleted.')
    


def find_cell(image):
    """ Identifies cells in range of lower and upper blue RGB color spectrum.

    Args:
        image (numpy.ndarray): Image loaded through cv2 package.

    Returns:
        [numpy.ndarray]: Returns blue cell mask.
    """
    
    # RGB type blue color interval
    lower_blue = np.array([90,80,160])
    upper_blue = np.array([150,140,255])
    
    # Convert image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Create a mask in range of upper and lower blue
    mask = cv2.inRange(image, lower_blue, upper_blue)

    return mask



def crop_image(path):
    """ Crops image around masked part. Mask contains a region of interest 
    (A white cell).

    Args:
        path (string): String which contains image path.

    Returns:
        [numpy.ndarray]: Cropped image in a black frame.
    """

    image = cv2.imread(path)
    mask = find_cell(image)
    #plt.imshow(mask, cmap='gray')
    #plt.show()

    # Find mask edges
    indices = np.nonzero(mask)
    size = len(set(zip(indices[0], indices[1])))
    avgX, avgY = 0, 0
    minX, minY = 10000, 10000
    maxX, maxY = -1, -1

    # Resize croped image dynamically
    for (x, y) in set(zip(indices[0], indices[1])):
        
        minX, minY, maxX, maxY = min(minX, x), min(minY, y), max(maxX, x), max(maxY, y)
        avgX += x
        avgY += y

    if size != 0:
        avgX /= size
    if size != 0:
        avgY /= size

    cropped = image[minX:maxX, minY:maxY]
    
    height, width = (333,333)

    frame = np.zeros((400,400, 3), np.uint8)
    x_offset = int((width - cropped.shape[1])/2)
    y_offset = int((height - cropped.shape[0])/2)

    frame[y_offset:y_offset+cropped.shape[0], x_offset:x_offset+cropped.shape[1]] = cropped

    return frame


def run(input_path, output_path):
    """ Collects all functions, iterates through a folder structure, crops images from 
    input path and saves them to output file.

    Args:
        input_path (string): Input images for cropping.
        output_path (string): Output path for saving cropped images.
    """

    #remove_if_exists(output_path)
    #create_directory(output_path)
    

    print('Processing', input_path)
    for path in glob.iglob(input_path + '*.jpeg'):
        
        #print('Processing', path)
        image = crop_image(path)
        out = output_path + os.path.basename(path)
        
        # Skips generated empty black (numpy.zeros) masks, error prevention.
        try:
            cv2.imwrite (out, image)
        except:
            print('Couldn\'t find a mask! Moving on.')
            pass

#run(input_path_eosino, output_path_eosino)
#run(input_path_lympho, output_path_lympho)
#run(input_path_mono, output_path_mono)
#run(input_path_neutro, output_path_neutro)

## Stvaranje modela

In [ ]:

%tensorflow_version 2.x

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, MaxPool2D, Flatten
from keras.preprocessing import image
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

generator = image.ImageDataGenerator(
        rescale = 1./255,
        featurewise_center=False,           # set input mean to 0 over the dataset
        samplewise_center=False,            # set each sample mean to 0
        featurewise_std_normalization=False,# divide inputs by std of the dataset
        samplewise_std_normalization=False, # divide each input by its std
        zca_whitening=False,                # apply ZCA whi0tening
        rotation_range=10,                  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,              # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,             # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,               # randomly flip images
        vertical_flip=False)

dataset_default = generator.flow_from_directory(
    shuffle = True,
    batch_size = 32,
    target_size = (80, 80),
    directory = '/content/drive/My Drive/Projects/Blood_Cells_Dataset/blood_cell_dataset/dataset2-master/dataset2-master/images/TRAIN/'
)

dataset_processed = generator.flow_from_directory(
    shuffle = True,
    batch_size = 32,
    target_size = (80, 80),
    directory = '/content/drive/My Drive/Projects/Blood_Cells_Dataset/blood_cell_dataset/dataset2-master/dataset2-master/images/TRAIN_PROCESSED/'
)


def model():
    model = Sequential()
    model.add(Conv2D(80, (3,3), strides = (1, 1), activation = 'relu'))
    model.add(Conv2D(64, (3,3), strides = (1, 1), activation = 'relu', input_shape = (80, 80, 3)))
    model.add(MaxPool2D(pool_size = (2,2)))
    model.add(Conv2D(64, (3,3), strides = (1,1), activation = 'relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())

    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation = 'softmax'))

    model.compile(loss = 'categorical_crossentropy', optimizer = 'adadelta', metrics = ['accuracy'])
    
    return model





Found 9957 images belonging to 4 classes.
Found 9960 images belonging to 4 classes.


## Treniranje default modela

In [ ]:
#model_default = model()
#model_default.fit(dataset_default, steps_per_epoch = None, epochs = 30, verbose = 1)
#model_default.save('/content/drive/My Drive/Projects/Blood_Cells_Dataset/model_default.h5')


Epoch 1/30
312/312 [==============================] - 579s 2s/step - batch: 155.5000 - size: 31.9135 - loss: 1.3916 - accuracy: 0.2548
Epoch 2/30
312/312 [==============================] - 577s 2s/step - batch: 155.5000 - size: 31.9135 - loss: 1.3899 - accuracy: 0.2491
Epoch 3/30
312/312 [==============================] - 580s 2s/step - batch: 155.5000 - size: 31.9135 - loss: 1.3876 - accuracy: 0.2534
Epoch 4/30
312/312 [==============================] - 580s 2s/step - batch: 155.5000 - size: 31.9135 - loss: 1.3874 - accuracy: 0.2566
Epoch 5/30
312/312 [==============================] - 580s 2s/step - batch: 155.5000 - size: 31.9135 - loss: 1.3863 - accuracy: 0.2599
Epoch 6/30
312/312 [==============================] - 579s 2s/step - batch: 155.5000 - size: 31.9135 - loss: 1.3869 - accuracy: 0.2556
Epoch 7/30
312/312 [==============================] - 580s 2s/step - batch: 155.5000 - size: 31.9135 - loss: 1.3866 - accuracy: 0.2558
Epoch 8/30
312/312 [==============================] - 5

## Treniranje procesiranog modela

In [ ]:
#model_default = model()
#model_default.fit(dataset_processed, steps_per_epoch = None, epochs = 30, verbose = 1)
#model_default.save('/content/drive/My Drive/Projects/Blood_Cells_Dataset/model_processed.h5')


Epoch 1/30
312/312 [==============================] - 3748s 12s/step - batch: 155.5000 - size: 31.9231 - loss: 1.3834 - accuracy: 0.2537
Epoch 2/30
312/312 [==============================] - 496s 2s/step - batch: 155.5000 - size: 31.9231 - loss: 1.3760 - accuracy: 0.2758
Epoch 3/30
312/312 [==============================] - 495s 2s/step - batch: 155.5000 - size: 31.9231 - loss: 1.3735 - accuracy: 0.2791
Epoch 4/30
312/312 [==============================] - 495s 2s/step - batch: 155.5000 - size: 31.9231 - loss: 1.3719 - accuracy: 0.2890
Epoch 5/30
312/312 [==============================] - 494s 2s/step - batch: 155.5000 - size: 31.9231 - loss: 1.3691 - accuracy: 0.2965
Epoch 6/30
312/312 [==============================] - 494s 2s/step - batch: 155.5000 - size: 31.9231 - loss: 1.3689 - accuracy: 0.2993
Epoch 7/30
312/312 [==============================] - 494s 2s/step - batch: 155.5000 - size: 31.9231 - loss: 1.3677 - accuracy: 0.3111
Epoch 8/30
312/312 [==============================] -

## Testiranje default modela

### Testiranje default modela na jednostavnom skupu za testiranje

In [12]:
from keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import glob                         

classes = ['EOSINOPHIL','LYMPHOCYTE','MONOCYTE','NEUTROPHIL']
model = load_model('/content/drive/My Drive/Projects/Blood_Cells_Dataset/model_processed.h5')


def predict_model(cell, dataset):

  cell = cell.lower()
  cell_count = 0
  data_counter = 0

  print('\nProcessing: ' + cell)
  for path in glob.iglob('/content/drive/My Drive/Projects/Blood_Cells_Dataset/blood_cell_dataset/dataset2-master/dataset2-master/images/' + dataset.upper() + '/' + cell.upper() + '/' + '*.jpeg'):
      
      #print(path)
      img = image.load_img(path, target_size=(80,80))
      img = image.img_to_array(img)

      img/=255
      img = img.reshape(1, 80, 80, 3)
      prediction = model.predict(img)
      #print(prediction)

      class_name = classes[np.argmax(prediction)]
      #print(class_name)

      if class_name == cell.upper():
        cell_count += 1
      
      data_counter += 1

  print('Files in '+ dataset.upper() +' folder:', data_counter)
  print(cell.capitalize() +' count: ' + str(cell_count))
  print('Correctly classified: ' + str((cell_count/data_counter)*100) + str('%'))

#predict_model('eosinophil', 'TEST_SIMPLE')
predict_model('lymphocyte', 'TEST_SIMPLE')
#predict_model('monocyte', 'TEST_SIMPLE')
#predict_model('neutrophil', 'TEST_SIMPLE')

    



Processing: lymphocyte
Files in TEST_SIMPLE folder: 6
Lymphocyte count: 6
Correctly classified: 100.0%
